In [19]:
from scipy import spatial, sparse
from collections import Counter
from sklearn.externals import joblib 

import os
import imp
import gzip
import copy
import nltk
import pickle
import scipy
import datetime

import numpy as np
import pandas as pd

# Measures

In [3]:
def coherence_score(X, topic_sentiment_df):
    X[X>1] = 1    
    totalcnt = len(topic_sentiment_df)
    total = 0
    for allwords in topic_sentiment_df:
        for word1 in allwords:
            for word2 in allwords:
                if word1 != word2:
                    ind1 = vocabulary[word1]
                    ind2 = vocabulary[word2]
                    total += np.log((np.matmul(X[:,ind1].T, X[:,ind2]) + 1.0)/np.sum(X[:,ind2]))
    return total/(2*totalcnt)

In [4]:
def kl_score(pk,qk):
    return (scipy.stats.entropy(pk,qk)*.5 + scipy.stats.entropy(qk,pk)*.5)

# Evaluation

In [6]:
sampler = joblib.load("dumps/2019-07-27 09:26:01.486138/sampler_electronics_5k_fold_4_maxiter_20_iter_4_in_5")

In [7]:
t_words = sampler.getTopKWords(5, sampler.words)
top_words = [t_words[i] for i in t_words.keys()]
document_topic = sampler.theta().argmax(axis=1)

In [8]:
coherence_score(sampler.get_count_matrix(), top_words)

NameError: global name 'vocabulary' is not defined

In [22]:
def get_hscore(dt_distribution, X, k):
    testlen = X.shape[0]
    all_kl_scores = np.zeros((testlen, testlen))
    for i in range(testlen-1):
        for j in range(i+1,testlen):
            score = kl_score(dt_distribution[i],dt_distribution[j])
            all_kl_scores[i,j] = score
            all_kl_scores[j,i] = score
    dt = np.zeros((X.shape[0], k))

    for i in range(X.shape[0]):
        dt[i, dt_distribution[i].argmax()]=1

    intradist = 0
    for i in range(k):
        cnt = dt[:,i].sum()
        tmp = np.outer(dt[:,i],dt[:,i])
        tmp = tmp * all_kl_scores
        intradist += tmp.sum()*1.0/(cnt*(cnt-1))
    intradist = intradist/k

    interdist = 0
    for i in range(k):
        for j in range(k):
            if i != j:
                cnt_i = dt[:,i].sum()
                cnt_j = dt[:,j].sum()
                tmp = np.outer(dt[:,i], dt[:,j])
                tmp = tmp * all_kl_scores
                interdist += tmp.sum()*1.0/(cnt_i*cnt_j)

    interdist = interdist/(k*(k-1))
    return intradist/interdist

In [23]:
%%time
get_hscore(sampler.theta(), sampler.get_count_matrix(), sampler.n_topics)

CPU times: user 8min 16s, sys: 15.6 s, total: 8min 32s
Wall time: 8min 21s


0.2633642071223209

In [ ]:
top_words

# Entropy Calculations

In [ ]:
idf_matrix = (sampler.get_df_matrix() > 0).astype(int)

In [ ]:
word_freq = idf_matrix.sum(axis=0)

In [ ]:
idf_matrix = idf_matrix * word_freq
idf_matrix = idf_matrix * 1.0/idf_matrix.shape[0]
document_topic_entropy = scipy.stats.entropy(sampler.nmz.transpose())

In [ ]:
word_len = sampler.get_count_matrix().sum(axis=1)

In [ ]:
print (np.corrcoef(document_topic_entropy, word_len))

In [ ]:
document_popularity = np.true_divide(idf_matrix.sum(1),(idf_matrix != 0).sum(1)) #scipy.stats.mstats.gmean(idf_matrix,axis=1)
print (np.corrcoef(document_topic_entropy, document_popularity))

In [ ]:
normalized_dts = sampler.nmzs * sampler.nmz[:,:,np.newaxis]
normalized_dts /= normalized_dts.sum(axis=-1)[:,:,np.newaxis]

In [ ]:
document_topic_sentiment_crossentropy = np.array([[scipy.stats.entropy(j) for j in i] for i in normalized_dts])
document_ts_entropy_min = document_topic_sentiment_crossentropy.min(axis=1)
document_ts_entropy_mean = document_topic_sentiment_crossentropy.mean(axis=1)
document_ts_entropy_var = np.sqrt(document_topic_sentiment_crossentropy.var(axis=1))

In [ ]:
document_ts_entropy_var

In [ ]:
# idf_matrix = (sampler.wordOccuranceMatrix > 0).astype(int)
# word_freq = idf_matrix.sum(axis=0)
# idf_matrix = idf_matrix * word_freq
# idf_matrix = idf_matrix * 1.0/idf_matrix.shape[0]
# document_topic_entropy = scipy.stats.entropy(sampler.dt_distribution.transpose())
# word_len = sampler.wordOccuranceMatrix.sum(axis=1)
# print (np.corrcoef(document_topic_entropy, word_len))
# #document_topic_entropy_len_normalized = document_topic_entropy * np.sqrt(word_len/2)
# document_popularity = np.true_divide(idf_matrix.sum(1),(idf_matrix != 0).sum(1)) #scipy.stats.mstats.gmean(idf_matrix,axis=1)
# print (np.corrcoef(document_topic_entropy, document_popularity))
# normalized_dts = sampler.dts_distribution * sampler.dt_distribution[:,:,np.newaxis]
# normalized_dts /= normalized_dts.sum(axis=-1)[:,:,np.newaxis]
# document_topic_sentiment_crossentropy = np.array([[scipy.stats.entropy(j) for j in i] for i in normalized_dts])
# document_ts_entropy_min = document_topic_sentiment_crossentropy.min(axis=1)
# document_ts_entropy_mean = document_topic_sentiment_crossentropy.mean(axis=1)
# document_ts_entropy_var = np.sqrt(document_topic_sentiment_crossentropy.var(axis=1))